# MACHINE LEARNING PARA CALCULAR A EVASÃO DOS ALUNOS DO PERÍODO HÍBRIDO

#### UTILIZANDO CATBOOST

##### TODAS AS IMPORTAÇÕES NESCESSÁRIAS PARA O PROJETO

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pickle


##### IMPORTANDO A TABELA DO EXEL, QUE NELA CONTEM OS DADOS DOS ALUNOS VINDO DO BANCO DE DADOS

In [ ]:
df = pd.read_excel("Base_dados.xlsx", usecols=['EVADIDO','DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP','MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS'], engine = "openpyxl")
df = df.fillna('n')
print(df.columns)

##### DECLARANDO TODAS AS COLUNAS DO EXEL QUE EU VOU USAR, E TRANSFORMANDO AS COLUNAS COM DADOS DE "SIM" OU "NÃO" PARA BINÁRIAS

In [ ]:
colunas = ['EVADIDO','DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP','MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS']


df[colunas] = df[colunas].fillna("SEM VALOR").astype(str).apply(lambda x: x.str.strip().str.upper())

# Convertendo binários de 'S' para 1 e outros para 0
df['DP'] = df['DP'].apply(lambda valor: 1 if valor == 'S' else 0)
df['EVADIDO'] = df['EVADIDO'].apply(lambda valor: 1 if valor == 'S' else 0)
df['INADIMPLENTE'] = df['INADIMPLENTE'].apply(lambda valor: 1 if valor == 'S' else 0)

# Convertendo colunas numéricas de volta ao tipo correto
df['QTD_MESES_INADI'] = pd.to_numeric(df['QTD_MESES_INADI'], errors='coerce').fillna(0).astype(int)
df['QTD_DP'] = pd.to_numeric(df['QTD_DP'], errors='coerce').fillna(0).astype(int)
df['MEDIA_DP_POR_INADI'] = pd.to_numeric(df['MEDIA_DP_POR_INADI'], errors='coerce').fillna(0).astype(float)
df['MEDIA_COBRANCAS'] = pd.to_numeric(df['MEDIA_COBRANCAS'], errors='coerce').fillna(0).astype(float)
df['PERFIL_RISCO'] = pd.to_numeric(df['PERFIL_RISCO'], errors='coerce').fillna(0).astype(float)
df['QTD_PERIODOS'] = pd.to_numeric(df['QTD_PERIODOS'], errors='coerce').fillna(0).astype(float)
df['CURSO'] = pd.to_numeric(df['CURSO'], errors='coerce').fillna(0).astype(float)


print(df)
print(df[colunas].dtypes)

##### DECLARO AS COLUNAS DE APOIO, AS FEATURES (X), E A COLUNA DE BASE (Y)

##### SEPARO OS DADOS DE TREINO DO X E DO Y

In [ ]:
X = df[['DP','INADIMPLENTE','QTD_MESES_INADI','CURSO','QTD_DP','MEDIA_DP_POR_INADI','PERFIL_RISCO','MEDIA_COBRANCAS','QTD_PERIODOS']]
y = df['EVADIDO']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


##### BALENCEIO TODOS OS DADOS COM O COMANDO <I><B>SMOTE</I></B>

In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

##### UTILIZO O MODELO DE CATBOOST PARA MINHA MACHINE, E APLICO ESSE MODELO PARA RODAR COM MEUS DADOS

In [ ]:
model = CatBoostClassifier(verbose=0, random_state=42)


model.fit(X_train, y_train)

y_pred_all = model.predict(X)

print(classification_report(y, y_pred_all))

##### FAÇO UM FOR , PARA TESTAR CADA THRESHOLD DE DECISÃO, VER QUAL DE 0.1 A 1.0 É MELHOR PARA OS MEUS DADOS

###### SE NESCESSÁRIOS, TESTAR EM VÁRIAS CASAS DECIMAIS , DEPENDENDO DA PRECISÃO QUE SEUS DADOS TEM, COMO POR EXEMPLO 0.01, 0.001 ETC

In [ ]:

pr_table = []
print("\n### Resultados CatBoost com melhores params ###\n")

# Loop para testar vários thresholds de decisão
for thresh in [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]:
  
    y_pred = (y_pred_all > thresh).astype(int)
    print(f"--- Threshold: {thresh} ---")
    print(classification_report(y, y_pred))
    pr_table.append({
        'threshold': np.append(thresh, 1),
        'precision': float(classification_report(y, y_pred).split('\n')[3].split()[1]),
        'recall': float(classification_report(y, y_pred).split('\n')[3].split()[2])
    })


pr_table = pd.DataFrame(pr_table)

relevantes = pr_table[(pr_table['recall'] > 0.6) & (pr_table['precision'] > 0.5)]

relevantes = relevantes.sort_values(by='recall', ascending=False)

print(relevantes.head(10))

##### POR FIM, GERAR UMA MATRIZ DE CONFUSÃO, PARA DESCOBRIRMOS QUAIS PREVISÕES NOSSA MACHINE ACERTOU E ERROU

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = (y_pred_all > 0.5).astype(int)
matriz = confusion_matrix(y, y_pred) #y_test (valores reais) e y_pred (valores previstos)
sns.heatmap(matriz, annot=True, fmt='d', cmap='Purples',
            xticklabels=['Não Evadiu (previsão)', 'Evadiu (previsão)'],
            yticklabels=['Não Evadiu (real)', 'Evadiu (real)'])
plt.xlabel('Previsão')
plt.ylabel('Realidade')
plt.title('Matriz de Confusão(0.5)')
plt.show()

##### GRAFICO A PARTE DO PROJETO, USADO PARA APENAS ALGUNS CASOS, PARA SABER AONDE ESTÁ O ENCONTRO DO RECALL COM O THERESHOLD, PARA DESCOBRIRMOS AONDE ESTÁ MELHOR ESTABILIDADE DO RECALL (MAIS ACERTOS), E O ENCONTRO DOS DOIS (F1-SCORE)

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


y_probs = model.predict_proba(X)[:, 1]

precision, recall, thresholds = precision_recall_curve(y, y_probs)

# Gráfico
plt.plot(thresholds, precision[:-1], 'b--', label='Precision')
plt.plot(thresholds, recall[:-1], 'g-', label='Recall')
plt.xlabel('Threshold de corte')
plt.ylabel('Score')
plt.title('Precision vs Recall vs Threshold')
plt.legend()
plt.grid(True)
plt.show()


### SALVA O MODELO, COM O THRESHOLD CALIBRADO

In [ ]:
import pickle


# Defina seu threshold
threshold = 0.5

# Empacote o modelo e o threshold em um dicionário
modelo_completo = {
    'modelo': model,
    'threshold': threshold
}

# Salve 
with open('modelo_lgb_com_threshold_05_real.pkl', 'wb') as f:
    pickle.dump(modelo_completo, f)
